### Functions are first-class objects in python

All data in python are represented as objects. First class functions are shorthand for Functions as first class objects.

First class objects can be defined as a program entity that:
 - is created at runtime
 - can be assigned to a variable or element in data structure
 - can be passed as an argument to a function
 - can be returned by a function
 
 Integers, strings, dicts are all first class objects
 
 
 Function object itself is an instance of the function class. type(function_name) -> function
 
 ### A variable pointing to a function and the function itself are 2 different concerns

In [1]:
def val(n):
  return n

type(val)

function

Functions can be assigned to a variable. The variable is now a reference to the function

Upon deleting the original function, only the reference is deleted and not the object itself. Another reference to the object would still work.

dunder name  would give the name assigned to the object upon initialization

In [2]:
def yell(text):
  return text.upper()
print(yell('Hello'))

hello = yell
print(hello('Hello'))

print('Deleting Yell')
del yell
print(hello('Hello'))
print(hello.__name__)
yell('Hello')



HELLO
HELLO
Deleting Yell
HELLO
yell


NameError: ignored

## Functions can be stored in Data Structures

In [0]:
lst = [hello, str.capitalize, str.lower]
print(lst)

for i in lst:
  print(i('Hello'))

##Functions can be passed to other functions. 
Eg: the map function 

In [0]:
print(list(map(hello, ('one', 'two', 'three'))))

def greet(func):
  print(func('Hello'))
  
greet(hello)

## Higher Order Functions:
A function that takes a function as an argument **or** returns a function as a result. 
Eg: map, sorted, filter, reduce

## Functions can be Nested

 - The function defined inside a function cannot be accessed outside the scope of the outside function
 - To do so, you have to return the function itself (without any arguments)
 - Hence functions can return behaviors as well through arguments
 - Inner functions can remember the variables in outer functions. However, if this variable is reassigned in the inner function, use global to declare the variable in the inner function. 

In [0]:
def outside_func():
  text = 'Hello'
  def inside_func():
    print(f'text inside: {text}')
  inside_func()
outside_func()
print('Calling Inside function')
inside_func()


##Method vs functions
- methods are dependant
- method is associated with a object. 

- Functions are independant block of code, caalled by its name
- Functions can be used defined or in-built (sum())

In [0]:
class C:
  def method_name(self):
    print('Method associated with class')

## Objects are bot callable but can be made callable

In [0]:
class Example(object):
  def __init__(self, val):
    self.val = val
    
  def __str__(self):
    return f'{self.val}'
  
eg = Example(4)
print(eg)
print('eg (object) is not callable. Calling eg() would throw an error')
callable(eg)

###To make an object callable use dunder call

In [0]:
class Example(object):
  def __init__(self, val):
    self.val = val
    
  def __call__(self):
    return self.val
  
eg = Example(6)
print(eg())
print(callable(eg))
eg.__dict__

##Modern replacement for Map, filter
A listcomp or genexp does the job of map and filter combined and is more readable

In [0]:
def factorial(n):
  return 1 if n < 2 else n * factorial(n-1)

fact = factorial
fact(5)

In [0]:
print(f'Map: {list(map(fact, range(6)))}')
print(f'List comp: {[fact(n) for n in range(6)]}')

print(f'Map and Reduce: {list(map(fact, filter(lambda n:n%2, range(6))))}')
print(f'List comp: {[fact(n) for n in range(6) if n%2]}')

In python3 reduce was demoted from a built-in to the functools module.
Its most common use case **summation**  is better served using sum() without any imports required

In [0]:
from functools import reduce
from operator import add
print(f'Reduce: {reduce(add, range(6))}')
print(f'Sum: {sum(range(6))}')

###all() and any() are other built-in reducables.

##Anonymous functions - Lambda
- a function without a name
 - lambda keyword creates an anonymous function with python exp
 -  lambda's body cannot make assignments or use python while, try, etc statements since they are **pure expressions**
 

In [0]:
friends = ['ajay', 'akash', 'priya', 'vaishali']
print(sorted(friends, key=lambda x:x[::-1])) # sorting words by reversing each individual word

##Seven flavours of callable objects

() can be applied to objects beyonf functions. This can be tested via callable(object)

There are 7 callable types:
 - User defined functions
 - in built functions (len())
 - built in methods (dict.get()) 
 - methods defined in a class
 - classes (when invoked runs dunder new to create an instance and dunder init to initialize it)
 - class instances (by defining dunder class on the class)
 - generator functions (functions or methods that use yield keyword. return generator object)

##Function Introspection
Function objects have mave attributes. Most of these are common to Pyhton objects in general

In [0]:
def factorial(n):
  x = 10
  return 1 if n < 2 else n * factorial(n-1)
fact = factorial
dir(factorial)

###Attributes specific to functions
Let us look at the attributes specific to functions and not generic objects in Python

In [0]:
class C: pass
obj = C()
def func(): pass
sorted(set(dir(func)) - set(dir(obj))) 

##** dunder defaults ** 

within a function object, **dunder defaults** holds  a tuple with the default values of positional and keyword arguments. 

The defaults for keyword only argumnets appear in **dunder kwdefaults**

The names of these arguments are foud in **dunder code** attribute

In [0]:
def func(a, *args, k1=100, **kwargs):
  args_list = list(args)
  args_list.append(a)
  kwargs['k1'] = k1
  print(f'Arguments: {args_list}')
  print(f'Keyword arguments: {[(key, val) for key, val in kwargs.items()]}')
        
f1 = func(10, 20, 30, k2=50)

In [0]:
print(func.__defaults__)
print(func.__code__.co_varnames)
print(func.__code__.co_argcount)


As we can see, the arrangement is quite inconvenient. 
- __code__.co_varnames has not only the arguments but also the local variables created in the function(Just names)
- __code__.co_argcount give the gives the arguments count - **N**. 
- The first N elements in co_varnames gives the argument names.
- Again, to get the keyword arguments, we have to compare with dunder default. 
A better way is to use the **inspect** module

In [0]:
from inspect import signature
sig = signature(func)
print(str(sig))
for name, param in sig.parameters.items():
  print(f'{name}={param.default}')

inspect.signature returns a signature object which has a parameters attribute (an ordered mapping) of names to inspect
- inspect._empty  denotes attributes with no default

## Function Annotations
Helps attach metadata to the parameters of a function declaration and its return value

In [0]:
def func(a, *args, k1:'int > 0'=100, **kwargs) -> str:
  args_list = list(args)
  args_list.append(a)
  kwargs['k1'] = k1
  print(f'Arguments: {args_list}')
  print(f'Keyword arguments: {[(key, val) for key, val in kwargs.items()]}')
  return a
f1 = func(10, 20, 30, k2=50)
print(f1, type(f1))
func.__annotations__

As you can see, the annotations are stored in dunder annotations. However there are no checks, enforcements, validations or any actions performed. the function returns an int value despite stating its return value to be str. However, throws no error. 

## Packages for functional programming
There is no equivalent for multiply like sum(). mul() can only take 2 arguments. To multiply a number of items (like calculating factorial), we have to use reduce

In [0]:
from operator import mul
from functools import reduce

print(f'Using lambda {reduce(lambda a,b: a*b, range(1, 6))}')
print(f'Using mul from operators {reduce(mul, range(1,6))}')

Another place where lambda can be replaced would be using **itemgetter** and **attrgetter**

On passing multiple index args to itemgetter, it builds a function which returns tuples with extracted values

##itemgetter
- used to access square bracket elements like list, dictionary, etc

In [4]:
from operator import itemgetter

def itemggetter(*items):
  if len(items) == 1:
    item =  items[0]
    def g(obj):
      return obj[item]
  else:
    def g(obj):
      return tuple(obj[item] for item in items)
  return g
    
print(itemgetter(1)('ABCDEF'))
print(itemgetter(1, 4)('ABCDEF'))


name = {'fname': 'gajal', 'lname': 'agarwala'}
print(itemgetter('lname')(name))

metro_data = [
  ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
  ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
  ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
  ('New York-Newark', 'MX', 20.104, (40.808611, -74.020386)),
  ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))]

print(sorted(metro_data, key=itemgetter(1)))

cc = itemgetter(1, 0)
for city in metro_data:
  print(cc(city))

B
('B', 'E')
agarwala
[('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)), ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)), ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)), ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)), ('New York-Newark', 'MX', 20.104, (40.808611, -74.020386))]
('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('MX', 'New York-Newark')
('BR', 'Sao Paulo')


##attrgetter
- creates functions to extract object attributes by name
- used on objects with . operator

In [17]:
from operator import attrgetter
    
class C:
  def __init__(self, a, b):
    self.a = a
    self.b = b
    
c = C(10, 20)
cc = attrgetter('a')
print(cc(c))

10


20

##methodcaller
- call a method by name on the object given as argument

In [53]:
from operator import methodcaller
a = ['llll omen', 'geoooom', 'skyooooo']
print([x.upper() for x in a])
mc = methodcaller('upper')
print(list(mc(x) for x in a))
print(sorted(a, key=methodcaller('index', 'o'))) # 'o' is the additional argument that index takes
print(sorted(a, key=lambda x: x.count('o')))
print(sorted(a, key=methodcaller('count', 'o')))

['LLLL OMEN', 'GEOOOOM', 'SKYOOOOO']
['LLLL OMEN', 'GEOOOOM', 'SKYOOOOO']
['geoooom', 'skyooooo', 'llll omen']
['llll omen', 'geoooom', 'skyooooo']
['llll omen', 'geoooom', 'skyooooo']


##Freezing arguments with functools.partial
- a higher order function that allows partial application of a function
- given a function, a partial application produces a new callable with some of the arguments of the original function fixed
- For example, mul cannot be used with map. But a partial function triple can be used to replace mul. 

In [58]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
print(triple(7))

print(list(map(triple, range(10))))

21
[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]
